In [1]:
import os, sys
root_dir = os.path.dirname(os.getcwd())
sys.path.append(root_dir)

In [2]:
# Plotting bar plots for each cluster
clusters = [
    'Color-I', 'Color-II', 
    'Count-I', 'Count-II',
    'Style-I', 'Style-II',
    'Action-I', 'Action-II',
    'Background-I', 'Background-II',
]

model_str = {
    'Emu': 'Emu',
    'SEED-LLaMA': 'SEED',
    'GILL': 'gill'
}

In [ ]:
def get_level_image(model, level, metric, dataset_id, x, theta):
    results_dir = os.path.join(root_dir, 'results')
    detail_folders = [
        os.path.join(results_dir, f'{metric[0]}_evaluation/detail'),
        os.path.join(results_dir, f'{metric[0]}_evaluation_m/detail')
    ]
    for detail_folder in detail_folders:
        for file in os.listdir(detail_folder):
            if model in file.lower() and f'[{dataset_id}]' in file and not '(' in file:
                
    
    